# Using BeautifulSoup4 to scrape IMDb website for budget and metascore

In [ ]:
conda install requests

In [8]:
conda install beautifulsoup4

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [22]:
df = pd.read_csv('C:/Users/Sophia/Documents/academy-awards/missingbudmeta.csv')

In [23]:
def getdeets(id1):
    URL = 'https://www.imdb.com/title/'+id1
    page=requests.get(URL)
    soup=BeautifulSoup(page.content, 'html.parser')
    ## getting metacritic score
    try:
        mscore = soup.find('div',class_='title-overview').find('div',id='title-overview-widget').find('div',class_='plot_summary_wrapper').find('div',class_='titleReviewBar').find('div',class_='titleReviewBarItem').find('a').text.strip()
    except AttributeError:
        mscore = 'nan'
    ## if no mscore, then it defaults to nan.
    ## if len(mscore)>3 then it has selected the next item, which is not the mscore. mscore is at most 100.
    if len(mscore)>3:
        mscore='nan'
    ## getting budget
    details = soup.find('div',id='titleDetails').find_all(class_='txt-block')
    budgettext = 0
    try:
        for x in details:
            if 'Budget' in x.text:
                budgettext = (x.text.strip().split('\n')[0].split(':')[1])
    except AttributeError:
        budgettext = 'nan'
    ## stripping budgettext string
    if type(budgettext)==str:
        budgettext = re.sub(r'[^\w\s]','',budgettext)
    else: budgettext=='nan'
    return mscore,budgettext
    

In [30]:
##testing on one id
a1,b1 = getdeets(df1.imdb_title_id[1])
print(a1)
print(b1)

53
200000000


In [34]:
## actually running on the dataframe df

df['mscore'] = 0
df['budg'] = 0
for ind,row in df.iterrows():
    id1 = row['imdb_title_id']
    mscore, budg = getdeets(id1)
    df['mscore'][ind] = mscore
    df['budg'][ind] = budg

C:\Users\Sophia\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Sophia\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Sophia\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs